In [35]:
import pandas as pd
import pickle
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [36]:
nn_model = pickle.load(open('model_Kaggle', 'rb'))

In [47]:
full_tripdata= pd.read_csv("data/test.csv")

In [48]:
full_tripdata.columns

Index([u'id', u'vendor_id', u'pickup_datetime', u'passenger_count',
       u'pickup_longitude', u'pickup_latitude', u'dropoff_longitude',
       u'dropoff_latitude', u'store_and_fwd_flag'],
      dtype='object')

In [49]:
#processing test data 

full_tripdata["pickup_datetime"] = pd.to_datetime(full_tripdata["pickup_datetime"])
full_tripdata = full_tripdata.drop(columns=['vendor_id','store_and_fwd_flag','passenger_count'])
full_tripdata["weekday"] = full_tripdata["pickup_datetime"].dt.dayofweek
full_tripdata["hourofday"] = full_tripdata["pickup_datetime"].dt.hour
#prepare location, use pre-trained k-mean model
model = pickle.load(open('location_cluster_kmean', 'rb'))
print full_tripdata.columns
pickupLocations = full_tripdata.iloc[:,2:4]
print pickupLocations.describe()
full_tripdata["pickupLoc"] = model.predict(pickupLocations.as_matrix())

dropoffLocations = full_tripdata.iloc[:,4:6]
full_tripdata["dropoffLoc"] = model.predict(dropoffLocations.as_matrix())
print dropoffLocations.describe()

training = full_tripdata.drop(columns=['pickup_datetime',
                                       'dropoff_longitude',
                                       'dropoff_latitude',
                                       'pickup_longitude',
                                       'pickup_latitude',
                                       'id'])

training['weekday'] = training['weekday'].astype('category')
training['hourofday'] = training['hourofday'].astype('category')
training['pickupLoc'] = training['pickupLoc'].astype('category')
training['dropoffLoc'] = training['dropoffLoc'].astype('category')

X = pd.get_dummies(training)

Index([u'id', u'pickup_datetime', u'pickup_longitude', u'pickup_latitude',
       u'dropoff_longitude', u'dropoff_latitude', u'weekday', u'hourofday'],
      dtype='object')
       pickup_longitude  pickup_latitude
count     625134.000000    625134.000000
mean         -73.973614        40.750927
std            0.073389         0.029848
min         -121.933128        37.389587
25%          -73.991852        40.737392
50%          -73.981743        40.754093
75%          -73.967400        40.768394
max          -69.248917        42.814938
       dropoff_longitude  dropoff_latitude
count      625134.000000     625134.000000
mean          -73.973458         40.751816
std             0.072565          0.035824
min          -121.933327         36.601322
25%           -73.991318         40.736000
50%           -73.979774         40.754543
75%           -73.963013         40.769852
max           -67.496796         48.857597


In [50]:
X.columns

Index([u'weekday_0', u'weekday_1', u'weekday_2', u'weekday_3', u'weekday_4',
       u'weekday_5', u'weekday_6', u'hourofday_0', u'hourofday_1',
       u'hourofday_2',
       ...
       u'dropoffLoc_40', u'dropoffLoc_41', u'dropoffLoc_42', u'dropoffLoc_43',
       u'dropoffLoc_44', u'dropoffLoc_45', u'dropoffLoc_46', u'dropoffLoc_47',
       u'dropoffLoc_48', u'dropoffLoc_49'],
      dtype='object', length=131)

In [51]:
data_test = xgb.DMatrix(X)
result = nn_model.predict(data_test)
#nn_model.feature_types

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130'] [u'weekday_0', u'weekday_

In [22]:
ytest = np.exp(result)

In [23]:
submission = pd.DataFrame({'id': test.id, 'trip_duration': ytest})

In [26]:
submission.to_csv("submission.csv",index=False)